# Introdution
1. This notebook demonstrate the use of transfer learning to achieve high accuracy.
2. The current version gives around 91% accuracy on leaderboard.
3. This notebook can be further improved to give higher accuracy.
4. My accuracy on leaderboard is 93.78 %.
5. The notebook is beginner friendly.
6. Dont forget to upvote if you like.

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('../input/hackereath-holiday-season-deep-learning-contest/dataset/train.csv')
df.head()

In [ ]:
sns.countplot(df['Class'])

In [ ]:
from keras.applications.inception_v3 import InceptionV3,preprocess_input

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
#It's good to preprocess images by the function provided by the model
datagen = ImageDataGenerator(
        preprocessing_function=preprocess_input)

In [ ]:
train_generator = datagen.flow_from_dataframe(
    df,
    directory='../input/hackereath-holiday-season-deep-learning-contest/dataset/train',
    x_col = 'Image',
    y_col = 'Class',
    target_size=(299,299),
    class_mode = 'categorical',
    batch_size=32)

In [ ]:
base_model = InceptionV3(include_top=False,weights='imagenet',input_shape=(299,299,3))

In [ ]:
base_model.trainable = False

In [ ]:
from keras import layers,models

In [ ]:
model = models.Sequential()
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(512,activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(256,activation='relu'))
model.add(layers.Dense(6,activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer='adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy'])

In [ ]:
train_steps = np.ceil(train_generator.n/train_generator.batch_size)

In [ ]:
model.fit(
    train_generator,
    epochs=12,
    batch_size=32,
    steps_per_epoch=train_steps)

In [ ]:
test_df = pd.DataFrame()
test_images = os.listdir('../input/hackereath-holiday-season-deep-learning-contest/dataset/test')
test_df['Image']=test_images
test_df.head()

In [ ]:
test_generator = datagen.flow_from_dataframe(
    test_df,
    directory='../input/hackereath-holiday-season-deep-learning-contest/dataset/test',
    x_col = 'Image',
    y_col = None,
    target_size=(299,299),
    class_mode = None,
    batch_size=32,
    shuffle = False)

In [ ]:
test_steps = np.ceil(test_generator.n/test_generator.batch_size)

In [ ]:
pred = model.predict_generator(
    test_generator,
    steps = test_steps,
    verbose = 1)

In [ ]:
pred = np.argmax(pred,axis=-1)

In [ ]:
test_df['Class']=pred

In [ ]:
num_to_class = dict((y,x) for (x,y) in train_generator.class_indices.items())
num_to_class

In [ ]:
test_df['Class']=test_df['Class'].map(num_to_class)
test_df.head()

In [ ]:
test_df.to_csv('pred.csv',index=False)

![](https://st3.depositphotos.com/1998651/13850/v/600/depositphotos_138506364-stock-illustration-cup-of-coffee-with-have.jpg)